In [ ]:
import numpy as np

# Score for each letter (b for blank tile)
LETTER_POINTS = {
"A":1, "B":3, "C":3, "D":2, "E":1, "F":4, "G":2, "H":4, "I":1,
"J":8, "K":5, "L":1, "M":3, "N":1, "O":1, "P":3, "Q":10, "R":1,
"S":1, "T":1, "U":1, "V":4, "W":4, "X":8, "Y":4, "Z":10, "b":0
}

# Load real Scrabble dictionary (ENABLE)
with open("enable1.txt", "r") as f:
    VALID_WORDS = set(w.strip().upper() for w in f)

# Functions
def reset_game():
    """Resets the letter and word multiplier"""

    # Ensure they are global variables
    global letter_multiplier, word_multiplier

    # The letter bonuses
    letter_multiplier = np.array([
        [1,1,1,2,1,1,1,1,1,1,1,2,1,1,1],
        [1,1,1,1,1,3,1,1,1,3,1,1,1,1,1],
        [1,1,1,1,1,1,2,1,2,1,1,1,1,1,1],
        [2,1,1,1,1,1,1,2,1,1,1,1,1,1,2],
        [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
        [1,3,1,1,1,3,1,1,1,3,1,1,1,3,1],
        [1,1,2,1,1,1,2,1,2,1,1,1,2,1,1],
        [1,1,1,2,1,1,1,1,1,1,1,2,1,1,1],
        [1,1,2,1,1,1,2,1,2,1,1,1,2,1,1],
        [1,3,1,1,1,3,1,1,1,3,1,1,1,3,1],
        [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
        [2,1,1,1,1,1,1,2,1,1,1,1,1,1,2],
        [1,1,1,1,1,1,2,1,2,1,1,1,1,1,1],
        [1,1,1,1,1,3,1,1,1,3,1,1,1,1,1],
        [1,1,1,2,1,1,1,1,1,1,1,2,1,1,1]
    ])

    # The word bonuses
    word_multiplier = np.array([
        [3,1,1,1,1,1,1,3,1,1,1,1,1,1,3],
        [1,2,1,1,1,1,1,1,1,1,1,1,1,2,1],
        [1,1,2,1,1,1,1,1,1,1,1,1,2,1,1],
        [1,1,1,2,1,1,1,1,1,1,1,2,1,1,1],
        [1,1,1,1,2,1,1,1,1,1,2,1,1,1,1],
        [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
        [3,1,1,2,1,1,1,2,1,1,1,2,1,1,3],
        [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
        [1,1,1,1,2,1,1,1,1,1,2,1,1,1,1],
        [1,1,1,2,1,1,1,1,1,1,1,2,1,1,1],
        [1,1,2,1,1,1,1,1,1,1,1,1,2,1,1],
        [1,2,1,1,1,1,1,1,1,1,1,1,1,2,1],
        [3,1,1,1,1,1,1,3,1,1,1,1,1,1,3]
    ])

def find_main_word(diff_indices, board):
    """Finds the main word that has been added"""

    # Normalize to list of tuples
    diff = [(int(r), int(c)) for r, c in diff_indices]
    rows = [r for r, c in diff]
    cols = [c for r, c in diff]

    # horizontal if all rows same
    if len(set(rows)) == 1:
        r = rows[0]
        start_c = min(cols)
        end_c = max(cols)
        # expand left
        while start_c > 0 and board[r, start_c - 1] != "":
            start_c -= 1
        # expand right
        while end_c < 14 and board[r, end_c + 1] != "":
            end_c += 1
        positions = [(r, c) for c in range(start_c, end_c + 1)]
        word = "".join(board[r, c] for (r, c) in positions)
        return word, positions

    # vertical if all cols same
    if len(set(cols)) == 1:
        c = cols[0]
        start_r = min(rows)
        end_r = max(rows)
        # expand up
        while start_r > 0 and board[start_r - 1, c] != "":
            start_r -= 1
        # expand down
        while end_r < 14 and board[end_r + 1, c] != "":
            end_r += 1
        positions = [(r, c) for r in range(start_r, end_r + 1)]
        word = "".join(board[r, c] for (r, c) in positions)
        return word, positions

    # not aligned: try to detect orientation from adjacency of the first new tile
    r0, c0 = diff[0]
    # horizontal adjacency?
    if (c0 > 0 and board[r0, c0 - 1] != "") or (c0 < 14 and board[r0, c0 + 1] != ""):
        start_c = c0
        while start_c > 0 and board[r0, start_c - 1] != "":
            start_c -= 1
        end_c = c0
        while end_c < 14 and board[r0, end_c + 1] != "":
            end_c += 1
        positions = [(r0, c) for c in range(start_c, end_c + 1)]
        word = "".join(board[r0, c] for (r0, c) in positions)
        return word, positions

    # vertical adjacency?
    if (r0 > 0 and board[r0 - 1, c0] != "") or (r0 < 14 and board[r0 + 1, c0] != ""):
        start_r = r0
        while start_r > 0 and board[start_r - 1, c0] != "":
            start_r -= 1
        end_r = r0
        while end_r < 14 and board[end_r + 1, c0] != "":
            end_r += 1
        positions = [(r, c0) for r in range(start_r, end_r + 1)]
        word = "".join(board[r, c0] for (r, c0) in positions)
        return word, positions

    # isolated single tile (no extension) -> return single letter word (will be invalid in VALID_WORDS)
    return board[r0, c0], [(r0, c0)]

def find_all_words(diff_indices, board):
    """Return a list of (word, positions) for all words formed this turn."""
    words = []
    main_word, main_pos = find_main_word(diff_indices, board)
    if main_word:
        words.append((main_word, main_pos))

    diff_set = set(diff_indices)
    for r, c in diff_indices:
        # Check perpendicular direction
        if all(pos[0] == r for pos in main_pos):
            # main word is horizontal → check vertical word
            start_r = r
            while start_r > 0 and board[start_r-1, c] != "":
                start_r -= 1
            end_r = r
            while end_r < 14 and board[end_r+1, c] != "":
                end_r += 1
            if end_r != start_r:  # there is a vertical word
                positions = [(rr, c) for rr in range(start_r, end_r+1)]
                word = "".join(board[rr, c] for rr in range(start_r, end_r+1))
                if (word, positions) not in words:
                    words.append((word, positions))
        else:
            # main word is vertical → check horizontal word
            start_c = c
            while start_c > 0 and board[r, start_c-1] != "":
                start_c -= 1
            end_c = c
            while end_c < 14 and board[r, end_c+1] != "":
                end_c += 1
            if end_c != start_c:  # there is a horizontal word
                positions = [(r, cc) for cc in range(start_c, end_c+1)]
                word = "".join(board[r, cc] for cc in range(start_c, end_c+1))
                if (word, positions) not in words:
                    words.append((word, positions))
    return words

def is_word_real(all_words):
    """Checks all words to ensure they are real"""

    for word, positions in all_words:
        # Skip validation if the word contains a blank tile (trust player)
        if "b" in word:
            continue

        # Check validity against the word list
        if word not in VALID_WORDS:
            print(f"'{word}' is not a valid word!")

    return

def calculate_score(all_words, diff_indices, current_board):
    """Calculates the score from the words in this turn"""

    # Extracting the words
    diff_set = set((int(r), int(c)) for r, c in diff_indices)

    # Finding the score of each word
    score = 0
    for word, positions in all_words:
        word_mult = []
        word_score = 0
        for r, c in positions:
            letter = current_board[r, c]
            base_points = LETTER_POINTS.get(letter, 0)
            if (r, c) in diff_set:
                base_points *= letter_multiplier[r, c]
                word_mult.append(word_multiplier[r, c])
            word_score += base_points
        score += word_score*max(word_mult)

    # Reset multipliers for the squares used in this turn
    for r, c in diff_set:
        letter_multiplier[r, c] = 1
        word_multiplier[r, c] = 1

    # Bingo bonus (for all seven tiles used in one turn)
    if len(diff_set) == 7:
        score += 50

    return score

def update_score(p1_score, p2_score, previous_board, current_board, turn):
    """Returns updated points for players"""
    
    # Find newly placed tiles
    new_word = previous_board != current_board
    diff_indices = list(zip(*np.where(new_word))) 
    all_words = find_all_words(diff_indices, current_board)

    # Validate that the words are real
    is_word_real(all_words)

    # Calculate score for all words
    score = calculate_score(all_words, diff_indices, current_board)

    # Find which player's score to add to
    if turn % 2 == 1:
        p1_score += score
    else:
        p2_score += score

    return p1_score, p2_score

def print_board(board):
    """Prints the Scrabble board"""

    # Print board
    print("    " + " ".join([f"{i+1:2}" for i in range(15)]))
    print("   " + "---" *15)
    for i, row in enumerate(board):
        line = "  ".join(letter if letter != "" else "." for letter in row)
        print(f"{i+1:2} | {line}")

In [ ]:
# Example Game
reset_game()
p1_score = 0; p2_score = 0
board0 = np.array([[""]*15 for _ in range(15)])
board1 = board0.copy()
board1[7,7] = "A"; board1[7,8] = "R"; board1[7,9] = "T"

# Turn 1
turn = 1
p1_score, p2_score = update_score(p1_score, p2_score, board0, board1, turn)
print_board(board1)
print(f"P1: {p1_score}pts, P2: {p2_score}pts.\n")

board2 = board1.copy()
board2[7,10] = "S"
board2[7,6] = "P" 

# Turn 2
turn = 2
p1_score, p2_score = update_score(p1_score, p2_score, board1, board2, turn)
print_board(board2)
print(f"P1: {p1_score}pts, P2: {p2_score}pts.\n")

board3 = board2.copy()
board3[4,10] = "K"
board3[5,10] = "I"
board3[6,10] = "T"

# Turn 3
turn = 3
p1_score, p2_score = update_score(p1_score, p2_score, board2, board3, turn)
print_board(board3)
print(f"P1: {p1_score}pts, P2: {p2_score}pts.\n")

board4 = board3.copy()
board4[6,9] = "I"
board4[8,9] = "b"

# Turn 4
turn = 4
p1_score, p2_score = update_score(p1_score, p2_score, board3, board4, turn)
print_board(board4)
print(f"P1: {p1_score}pts, P2: {p2_score}pts.\n")

     1  2  3  4  5  6  7  8  9 10 11 12 13 14 15
   ---------------------------------------------
 1 | .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 2 | .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 3 | .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 4 | .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 5 | .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 6 | .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 7 | .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 8 | .  .  .  .  .  .  .  A  R  T  .  .  .  .  .
 9 | .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
10 | .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
11 | .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
12 | .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
13 | .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
14 | .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
15 | .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
P1: 6pts, P2: 0pts.

     1  2  3  4  5  6  7  8  9 10 11 12 13 14 15
   ---------------------------------------------
 1 | .  .  .  .  .  .  .  .  .  .  .  .  .  .  .